# The hash locked contract
#### 06.2 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-01-12

* Write a Hash Time Locked Contract
* Use secret (hashed) passwords
* Use time delays
* Limits of passwords on the blockchain

## Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the 5 accounts and the Purestake credentials

In [ ]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [ ]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk import transaction
from algosdk.transaction import PaymentTxn
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.transaction import LogicSig, LogicSigTransaction

import algosdk.error
import json
import base64
import hashlib

In [ ]:
from pyteal import *

In [ ]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])
algod_client.status()["last-round"]

In [ ]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

#### Quick check of asset holdings, otherwise go to ...
- https://bank.testnet.algorand.network
- https://testnet.algoexplorer.io/dispenser

In [ ]:
round(algod_client.account_info(Alice['public'])['amount']/1e6,2)

## The Cash Machine revisited

#### Remember
* Alice created a *Cash Machine* contract that requires a password
* Bob has to know the password to withdraw money
* Alice must give the following to Bob
  * The **address** of the smart contract
  * The **code of the smart signature** $\leftarrow$ this is actually a **problem**

In [ ]:
# Looking again at the Cash Machine

# Step 1: Conditions as a PyTeal
cashmachine_pyteal = ( 
    Txn.note() == Bytes('{"Very long and very secret message"}') 
    )

# Step 2-3: Compile PyTeal -> Teal -> Bytecode for AVM
cashmachine_teal = compileTeal(cashmachine_pyteal, Mode.Signature, version=8)

# compile Teal -> Bytecode
Cashmachine = algod_client.compile(cashmachine_teal)
print("Alice sends these two items to Bob")
print("Compiled smart signature:", Cashmachine['result'])
print("Address of smart signature: ", Cashmachine['hash'])

#### So what exactly ...

is inside this smart signature code?

In [ ]:
# The code is bascially a base-64 encoding of the TEAL program
base64.b64decode(Cashmachine['result'])

## The Hash lock contract

#### Better
* Store the **hash** of the password

#### Step 1: Create a hash of the password

In [ ]:
secret_password = 'WSC secret'
hash = hashlib.sha256( secret_password.encode() )
print(hash.hexdigest())                                 # The hash is HEX encoded

#### Step 2: Use the hashed password in PyTeal condition

In [ ]:
hashlock_cond = ( 
    Sha256(Txn.note()) == Bytes("base16",hash.hexdigest()) 
    )

# prepare random condition
import random
a = Int( random.randrange(2**32-1) )
random_cond = ( a == a )

fee_cond = ( Txn.fee() <= Int(1000) )

safety_cond = And(
    Global.group_size() == Int(1),                   # Not part of a TX group
    Txn.type_enum() == TxnType.Payment,
    Txn.rekey_to() == Global.zero_address(),
    Txn.close_remainder_to() == Global.zero_address()
    )

hashlock_pyteal = And(
    hashlock_cond, 
    random_cond, 
    fee_cond, 
    safety_cond
    )

In [ ]:
# Alternatively if you have the hashed value in HEX = base16 representation
# Convert to Bytes using the following command
# Note: the "0x" at the beginning of the hash is optional

#hashlock_cond = (
#    Sha256(Txn.note()) == Bytes("base16","76241abc874467ec6cb2b9cdf60c712bfd213425fb391a5bef81b5efd22138d5")
#     )

# prepare random condition
import random
a = Int( random.randrange(2**32-1) )
random_cond = ( a == a )

fee_cond = ( Txn.fee() <= Int(1000) )

safety_cond = And(
    Global.group_size() == Int(1),                   # Not part of a TX group
    Txn.type_enum() == TxnType.Payment,
    Txn.rekey_to() == Global.zero_address(),
    )

hashlock_pyteal = And (hashlock_cond, random_cond, fee_cond, safety_cond)

#### Step 3-4: Compile

In [ ]:
# Compile PyTeal -> Teal -> Bytecode for AVM
hashlock_teal = compileTeal(hashlock_pyteal, Mode.Signature, version=8)
print(hashlock_teal)
print("")

# Step 3: Compile Teal 
Hashlock = algod_client.compile(hashlock_teal)
print("Alice sends these two items to Bob")
print("Compiled smart signature:", Hashlock['result'])
print("Address of smart signature: ", Hashlock['hash'])

#### Step 5: Quick check: can Bob crack the code?

In [ ]:
print(base64.b64decode(Hashlock['result']))
# BTW: can you find the hash of the password?
print(base64.b64decode(Hashlock['result']).hex())

#### Step 6: Alice is funding the Smart Signature

In [ ]:
# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(1.2*1e6)
txn = transaction.PaymentTxn(sender=Alice['public'], sp=sp, receiver=Hashlock['hash'], amt=amt)

# Step 2+3: sign and send
stxn = txn.sign(Alice['private'])
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

# The transaction does not give away any information
# The TEAL code is not even included
print('http://testnet.algoexplorer.io/tx/'+txid)

#### Step 7: Bob asks the smart signature to sign a transaction with the correct password

In [ ]:
# correct password
my_note        = 'WSC secret'

# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(1*1e6)              # <---------- OK!!
txn = PaymentTxn(sender=Hashlock['hash'], sp=sp, 
                 receiver=Bob['public'], amt=withdrawal_amt,
                 note = my_note.encode(),
                 close_remainder_to = Bob['public'])

# Step 2: sign TX <---- This step is different!
encodedProg = Hashlock['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

#### Step 8: Check the "Note" field on Algoexplorer
* The password is now out in the open ... we cannot do anything about it
* Hash lock contracts work exactly once

In [ ]:
print('http://testnet.algoexplorer.io/tx/'+txid)

## Things that do not work
* Using the wrong password
* Sending a hash of the pasword instead of the password (because it will be hashed again)